## File system

- When you install docker on a system, you get this folder structure: `var/lib/docker`
    - This will contain subdirs like `aufs`, `containers`, `image`, ...
    - This is where docker stores all its data by default


## Layered Architecture Advantage

- Recall from `5. docker images.ipynb` that Docker builds images as layers
- Every layer is stored as an increment from the previous layer

- What's the advantage of doing this? Imagine we have these 2 Dockerfiles    
    - File 1:
        ```docker
        FROM ubuntu

        RUN apt-get update && apt-get -y install python

        RUN pip install flask flask-mysql

        COPY . /opt/source-code

        ENTRYPOINT FLASK_APP=/opt/source-code/app.py flask run        
        ```
    
    - File 2:
        ```docker
        FROM ubuntu

        RUN apt-get update && apt-get -y install python

        RUN pip install flask flask-mysql

        COPY app2.py /opt/source-code

        ENTRYPOINT FLASK_APP=/opt/source-code/app2.py flask run        
        ```

- From these 2 files, the only difference is only in the last 2 commands 
    - Due to the layered building architecture, the first 3 layers are REUSED from the build of the first docker file, thereby saving time


- So what exactly happens in the build phase?
    - Step 1: Image layer. Everything before this point is frozen; no changes are allowed. To change this, you need to create a new build
        - Base Ubuntu layer
        - apt packages
        - pip packages
        - Source code copy
        - Define entry point
    
    - Step 2: Container layer. A container layer is created based off the layers from the previous step
        - This is a writeable layer
        - Stores data created by the container (log files, temporary files)
        - Only lasts as long as container is alive
        - If containers use the same image layer, it means they are sharing stuff in the image layer
        - If, for whatever reason, you want to modify the files in the image layer, Docker copies the file you want to modify onto the container layer and modifies it there. NOT the actual file from image layer
            - This is known as `copy on write`

## Persistent data storage

- Of course, while this image/container layer separation helps with the speed of the container spin up, it also has a glaring disadvantage; you cannot persist data easily in your container

- To do this, you need to use **volumes**

- Steps to set up persistent storage
    - `docker volume create some_persistent_volume`
        - This tells docker to create a persistent volume directory in the docker host, under `/var/lib/docker/volumes/some_persistent_volume`
    - `docker run -v some_persistent_volume:/var/lib/mysql mysql`   
        - Next, map the volume to the docker run command
        - This creates a new container, and mounts the data volume into the container
        - So all data written to the database lives on the docker host!
        - In fact, the `-v` will automatically do the `docker volume create` if it finds that the volume you specify does not exist yet!
    - If you have another data somewhere else in docker host that you want to use as the mount, simply provide the full path using the `-v` parameter
    
    - If you use the default volume approach, it is called **volume mounting**. If you specify yourown path, this is known as **bind mounting**

    - The more modern and also recommended syntax to do this is now `docker run --mount type=bind, source=/data/mysql, target=/var/lib/mysql mysql`

- More broadly, the **storage driver** is the component of docker responsible for maintaining the layered architecture, creating a writable layer, moving files etc

- Common storage drivers are
    - AUFS (default for Ubuntu)
    - ZFS
    - BTRFS
    - Device Mapper
    - Overlay
    - Overlay2